## 事前準備

1. Google Cloud の新しいプロジェクトを作成します。
2. Cloud Shell を開いて、次のコマンドで API を有効化します。
```
gcloud services enable \
  aiplatform.googleapis.com \
  notebooks.googleapis.com \
  cloudresourcemanager.googleapis.com

```
3. 次のコマンドで Vertex AI Workbench のインスタンスを作成します。
```
PROJECT_ID=$(gcloud config list --format 'value(core.project)')
gcloud workbench instances create agent-development \
  --project=$PROJECT_ID \
  --location=us-central1-a \
  --machine-type=e2-standard-2
```

4. クラウドコンソールのナビゲーションメニューから「Vertex AI」→「Workbench」を選択すると、作成したインスタンス agent-develpment があります。インスタンスの起動が完了するのを待って、「JUPYTERLAB を開く」をクリックしたら、「Python 3(ipykernel)」の新規ノートブックを作成します。

5. この後は、ノートブック上で以下のコマンドを実行していきます。

## ADK パッケージのインストール

In [ ]:
%pip install --upgrade --user \
    google-adk==0.5.0 \
    google-cloud-aiplatform==1.93.0

インストールしたパッケージを利用可能にするために、次のコマンドでカーネルを再起動します。

再起動を確認するポップアップが表示されるので [Ok] をクリックします。

In [9]:
import IPython
app = IPython.Application.instance()
_ = app.kernel.do_shutdown(True)

## Search Agent App の作成

In [1]:
import copy, datetime, json, os, pprint, time, uuid
import vertexai
from google.genai.types import Part, UserContent, ModelContent
from google.adk.agents.llm_agent import LlmAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools import google_search

[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
LOCATION = 'us-central1'

vertexai.init(project=PROJECT_ID, location=LOCATION,
              staging_bucket=f'gs://{PROJECT_ID}')

os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

In [2]:
instruction = '''
You are a friendly AI assistant that answers user's queries.
Use google_search to give answers based on the latest and objective information.

[Format instruction]
Output in Japanese, in plain text only.
Avoid adding citation marks such as [1][2].
'''

search_agent = LlmAgent(
    name='search_agent',
    model='gemini-2.0-flash-001',
    description='Agent to answer questions using Google Search.',
    instruction=instruction,
    tools=[google_search]
)

In [3]:
class LocalApp:
    def __init__(self, agent, app_name, user_id):
        self._agent = agent
        self._app_name = app_name
        self._user_id = user_id
        self._runner = Runner(
            app_name=self._app_name,
            agent=self._agent,
            artifact_service=InMemoryArtifactService(),
            session_service=InMemorySessionService(),
            memory_service=InMemoryMemoryService(),
        )
        self._session = self._runner.session_service.create_session(
            app_name=self._app_name,
            user_id=self._user_id,
            state={},
            session_id=uuid.uuid4().hex,
        )
        
    async def stream(self, query):
        content = UserContent(parts=[Part.from_text(text=query)])
        async_events = self._runner.run_async(
            user_id=self._user_id,
            session_id=self._session.id,
            new_message=content,
        )
        result = []
        async for event in async_events:
            if (event.content and event.content.parts):
                response = '\n'.join([p.text for p in event.content.parts if p.text])
                if response:
                    print(response)
                    result.append(response)
        return result

## 実行例

In [24]:
client = LocalApp(search_agent, 'Search Agent App', 'user00')

query = '''
高田馬場のおすすめのカレー屋は？
'''
_ = await client.stream(query)

 高田馬場でおすすめのカレー屋さんについて調べてみました。

 
 高田馬場にはたくさんのカレー屋さんがありますね。おすすめのお店としては、以下のようなところが挙げられます。

*   **カレーライス専門店ブラザー:** 鯖キーマカレーが人気のお店です。
*   **エチオピアカリーキッチン 高田馬場店:** 長時間煮込んだ野菜が美味しいトロトロカレーが楽しめます。
*   **プネウマカレー:** コスパが良いと評判のお店です。
*   **横浜ボンベイ 高田馬場店:** サラッとしたルーが特徴で、カシミールカレーが人気です。
*   **白カレーの店 1/f ゆらぎ:** 珍しい白カレーが味わえます。

この他にもたくさんのお店があるので、食べログやRettyなどのグルメサイトで調べてみるのもおすすめです。



SessionService が管理するセッション情報を確認します。

セッションを特定するのに必要な、`app_name`、`user_id`、`session_id` を確認します。

In [25]:
app_name = client._session.app_name
user_id = client._session.user_id
session_id = client._session.id

app_name, user_id, session_id

('Search Agent App', 'user00', 'e3d95fc4772f4992bf3ed35cfa209c80')

確認した情報を使って、セッションに保存された情報を確認します。

2 つのイベント（ユーザーの質問とエージェントの回答）が記録されています。

エージェントの回答を生成する際に用いた Google Serch の情報なども確認できます。

In [26]:
session = client._runner.session_service.get_session(
    app_name = app_name,
    user_id = user_id,
    session_id = session_id,
)

session

Session(id='e3d95fc4772f4992bf3ed35cfa209c80', app_name='Search Agent App', user_id='user00', state={}, events=[Event(content=UserContent(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='\n高田馬場のおすすめのカレー屋は？\n')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, invocation_id='e-5085e36c-66b0-4aac-b8c2-c2a3fee4d837', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='3on5aeKM', timestamp=1747603381.99736), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text=

次のように、これまでのやり取りが「イベント」として記録されています。

In [27]:
def format_timestamp(ts):
    return datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S.%f')

for event in session.events:
    print(format_timestamp(event.timestamp), event.author)

2025-05-18 21:23:01.997360 user
2025-05-18 21:23:01.998311 search_agent


特に、下記の要素から Google 検索に使ったキーワードが確認できます。

In [28]:
from IPython.display import display, HTML

search_queries = session.events[-1].grounding_metadata.web_search_queries
entry_point_html = session.events[-1].grounding_metadata.search_entry_point.rendered_content

print(search_queries)
display(HTML(entry_point_html))

['高田馬場 おすすめ カレー']


続けて質問すると、セッション情報を踏まえた回答が得られます。

In [29]:
query = '''
特に家族連れにおすすめなのは？
'''
_ = await client.stream(query)

 家族連れにおすすめの高田馬場のカレー屋さんについて調べてみます。


 家族連れにおすすめとなると、いくつかポイントがありますね。

*   **子供向けメニューやサービス:** 子供用のメニューがあるか、子供用椅子があるか、離乳食の持ち込みが可能かなどが重要です。
*   **お店の雰囲気:** 子連れでも気兼ねなく入れる雰囲気かどうかも大切です。
*   **座席:** ベビーカーを置けるスペースがあるか、子供が動きやすい座敷席があるかなども考慮したい点です。

これらの点を踏まえて、高田馬場でおすすめのカレー屋さんをいくつかご紹介します。

*   **プネウマカレー:** こちらのお店は、mamacoという子連れママ向けのグルメナビサイトで紹介されています。小さなお子様連れでもテイクアウトができるので、自宅でゆっくり味わうのも良いでしょう。
* **インド料理＆ファミリーレストラン ナマステヒマール:** 楽天ぐるなびによると、早稲田にありますが、子供連れOKとのことです。

食べログなどのグルメサイトでは、子連れで利用しやすいお店を検索できる機能があります。「高田馬場」と「子連れ」をキーワードに検索して、口コミやお店の情報を確認してみるのがおすすめです。



先ほどと同様にセッションに記録されたイベントを確認すると、新しい質問と回答のイベントが追加されています。

In [30]:
session = client._runner.session_service.get_session(
    app_name = app_name,
    user_id = user_id,
    session_id = session_id,
)

for event in session.events:
    print(format_timestamp(event.timestamp), event.author)

2025-05-18 21:23:01.997360 user
2025-05-18 21:23:01.998311 search_agent
2025-05-18 21:23:17.199571 user
2025-05-18 21:23:17.200245 search_agent


In [31]:
search_queries = session.events[-1].grounding_metadata.web_search_queries
entry_point_html = session.events[-1].grounding_metadata.search_entry_point.rendered_content

print(search_queries)
display(HTML(entry_point_html))

['高田馬場 カレー ファミリー', '高田馬場 カレー 子連れ', '高田馬場 レストラン 子連れ', '高田馬場 ディナー 子連れ', '高田馬場 カレー おすすめ']


## システムインストラクションの確認

ちょっとした裏技を使って、LlmAgent オブジェクトが Gemini API に送っている生の情報を覗いてみます。

In [32]:
import pprint
from google.adk.agents.invocation_context import InvocationContext
from typing import AsyncGenerator
from google.adk.events.event import Event
from google.adk.models.llm_request import LlmRequest

async def _run_one_step_async(
      self,
      invocation_context: InvocationContext,
  ) -> AsyncGenerator[Event, None]:
    """One step means one LLM call."""
    llm_request = LlmRequest()

    # Preprocess before calling the LLM.
    async for event in self._preprocess_async(invocation_context, llm_request):
        yield event
    if invocation_context.end_invocation:
        return

    # Calls the LLM.
    model_response_event = Event(
        id=Event.new_id(),
        invocation_id=invocation_context.invocation_id,
        author=invocation_context.agent.name,
        branch=invocation_context.branch,
    )

    ## DEBUG output
    if DEBUG:
        print('## Prompt contents ##')
        pprint.pp(llm_request.contents)
        print('----')
        print('## System instruction ##')
        print(llm_request.config.system_instruction)
        print('----')
        print('## Tools config ##')
        pprint.pp(llm_request.config.tools)
        print('----')
    ####

    async for llm_response in self._call_llm_async(
        invocation_context, llm_request, model_response_event
    ):
        # Postprocess after calling the LLM.
        async for event in self._postprocess_async(
            invocation_context, llm_request, llm_response, model_response_event
        ):
            yield event

from google.adk.flows.llm_flows.base_llm_flow import BaseLlmFlow
_run_one_step_async_original = copy.deepcopy(BaseLlmFlow._run_one_step_async)
BaseLlmFlow._run_one_step_async = _run_one_step_async

In [33]:
client = LocalApp(search_agent, 'Search Agent App', 'user00')

DEBUG = True
query = '''
高田馬場のおすすめのカレー屋は？
'''
_ = await client.stream(query)

## Prompt contents ##
[UserContent(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='\n高田馬場のおすすめのカレー屋は？\n')], role='user')]
----
## System instruction ##

You are a friendly AI assistant that answers user's queries.
Use google_search to give answers based on the latest and objective information.

[Format instruction]
Output in Japanese, in plain text only.
Avoid adding citation marks such as [1][2].


You are an agent. Your internal name is "search_agent".

 The description about you is "Agent to answer questions using Google Search."
----
## Tools config ##
[Tool(retrieval=None, google_search=GoogleSearch(), google_search_retrieval=None, enterprise_web_search=None, google_maps=None, code_execution=None, function_declarations=None)]
----
 高田馬場でおすすめのカレー屋について調べてみましょう。

 
 高田馬場にはたくさんのカレー屋さんがありますね。おすすめのお店をいくつかご紹介します。

 *   **横浜ボンベイ**: サラッとしたルーが特徴で、特に「カシミールカレー」が

In [34]:
query = '''
特に家族連れにおすすめなのは？
'''
_ = await client.stream(query)

## Prompt contents ##
[UserContent(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='\n高田馬場のおすすめのカレー屋は？\n')], role='user'),
 Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text=' 高田馬場でおすすめのカレー屋について調べてみましょう。\n\n '), Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text=' 高田馬場にはたくさんのカレー屋さんがありますね。おすすめのお店をいくつかご紹介します。\n\n *   **横浜ボンベイ**: サラッとしたルーが特徴で、特に「カシミールカレー」が人気です。辛口ですが、野菜の甘みとスパイスの爽快感が絶妙にマッチしています。\n *   **エチオピア**: 長時間煮込んだ野菜が美味しいカレーライス専門店です。\n *   **カレーライス専門店ブラザー**: 鯖キーマカレーが人気です。\n *   **プネウマカレー**: コスパの良いチキンカレーが人気です。\n *   **アプサラ**: スリランカカレーが楽しめます。\n\n 他にも、Rettyや食べログなどのサイトで多くのカレー店が紹介されていますので、参考にしてみてください。\n')]

`## Prompt contents ##` の部分を見ると、セッションに記録されたイベントをすべてプロンプトに含めるのではなく、コンテキスト（これまでの会話の流れ）を把握するのに必要な情報が選択されていることがわかります。

裏技で変更した内部関数を元に戻しておきます。

In [35]:
DEBUG = False
BaseLlmFlow._run_one_step_async = _run_one_step_async_original

## AgentEngine へのデプロイ

In [9]:
from vertexai import agent_engines

remote_agent = agent_engines.create(
    agent_engine=search_agent,
    requirements=[
        'google-adk==0.5.0',
        'google-cloud-aiplatform[agent_engines]==1.93.0',
    ]
)

Deploying google.adk.agents.Agent as an application.
Identified the following requirements: {'pydantic': '2.10.6', 'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.93.0'}
The following requirements are missing: {'pydantic', 'cloudpickle'}
The following requirements are appended: {'pydantic==2.10.6', 'cloudpickle==3.1.1'}
The final list of requirements: ['google-adk==0.5.0', 'google-cloud-aiplatform[agent_engines]==1.93.0', 'pydantic==2.10.6', 'cloudpickle==3.1.1']
Using bucket etsuji-15pro-poc
Wrote to gs://etsuji-15pro-poc/agent_engine/agent_engine.pkl
Writing to gs://etsuji-15pro-poc/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://etsuji-15pro-poc/agent_engine/dependencies.tar.gz
Creating AgentEngine
Create AgentEngine backing LRO: projects/879055303739/locations/us-central1/reasoningEngines/2370520681205989376/operations/4368320492692570112
View progress and logs at https://console.cloud.google.com/logs/query?project=etsuji-15pro-poc
A

In [10]:
class RemoteApp:
    def __init__(self, remote_agent, user_id):
        self._remote_agent = remote_agent
        self._user_id = user_id
        self._session = remote_agent.create_session(user_id=self._user_id)
    
    def _stream(self, query):
        events = self._remote_agent.stream_query(
            user_id=self._user_id,
            session_id=self._session['id'],
            message=query,
        )
        result = []
        for event in events:
            if ('content' in event and 'parts' in event['content']):
                response = '\n'.join(
                    [p['text'] for p in event['content']['parts'] if 'text' in p]
                )
                if response:
                    print(response)
                    result.append(response)
        return result

    def stream(self, query):
        # Retry 4 times in case of resource exhaustion 
        for c in range(4):
            if c > 0:
                time.sleep(2**(c-1))
            result = self._stream(query)
            if result:
                return result
            if DEBUG:
                print('----\nRetrying...\n----')
        return None # Permanent error

In [39]:
remote_client = RemoteApp(remote_agent, 'user00')

query = '''
今年のゴールデンウィークは、何連休でしょうか？
'''
_ = remote_client.stream(query)

 2025年のゴールデンウィークは、カレンダー通りに行くと、5月3日（土）から5月6日（火）までの4連休です。

ただし、有給休暇をうまく利用すれば、もっと長くすることも可能です。例えば、4月28日（月）、4月30日（水）、5月1日（木）、5月2日（金）に有給休暇を取得すると、4月26日（土）から5月6日（火）までの12連休にすることができます。また、5月1日（木）と5月2日（金）に有給休暇を取得すると、4月29日（火）から5月6日（火）までの7連休にすることができます。



In [40]:
query = '''
来年はどうなりますか？
'''
_ = remote_client.stream(query)

 2026年のゴールデンウィークは、5月2日（土）から5月6日（水）までの5連休です。4月30日（木）と5月1日（金）に有給休暇を取得すると、4月29日（水）から5月6日（水）までの最大8連休にすることも可能です。



セッション情報を参照する際は、`VertexAiSessionService` を使用します。

In [41]:
app_name = remote_client._session['app_name']
user_id = remote_client._session['user_id']
session_id = remote_client._session['id']

app_name, user_id, session_id

('100143519057838080', 'user00', '5906942052028907520')

In [42]:
from google.adk.sessions import VertexAiSessionService

session_service = VertexAiSessionService(
    project = PROJECT_ID,
    location = LOCATION,
)

session = session_service.get_session(
    app_name=app_name,
    user_id=user_id,
    session_id=session_id,
)

session

Session(id='5906942052028907520', app_name='100143519057838080', user_id='user00', state={}, events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='\n今年のゴールデンウィークは、何連休でしょうか？\n')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, invocation_id='e-536d602b-196c-494d-ae19-84e0692b5a77', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='9187233838428848128', timestamp=1747603552.023382), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, t

In [43]:
for event in session.events:
    print(format_timestamp(event.timestamp), event.author)

2025-05-18 21:25:52.023382 user
2025-05-18 21:25:52.100368 search_agent
2025-05-18 21:26:21.147094 user
2025-05-18 21:26:21.188163 search_agent


Event オブジェクトの構造がローカルで実行する場合と少し異なるので注意してください。

In [53]:
search_queries = session.events[1].grounding_metadata['webSearchQueries']
entry_point_html = session.events[1].grounding_metadata['searchEntryPoint']['renderedContent']

print(search_queries)
display(HTML(entry_point_html))

['2025年ゴールデンウィークは何連休', '2025 ゴールデンウィーク 祝日']


In [54]:
search_queries = session.events[3].grounding_metadata['webSearchQueries']
entry_point_html = session.events[3].grounding_metadata['searchEntryPoint']['renderedContent']

print(search_queries)
display(HTML(entry_point_html))

['2026 ゴールデンウィーク', 'ゴールデンウィーク 2026 何連休']


## 後片付け

デプロイ済みのエージェントをまとめて削除します。

In [13]:
for agent in agent_engines.list():
    print(agent.gca_resource.name)
    agent.delete(force=True)

projects/879055303739/locations/us-central1/reasoningEngines/2370520681205989376
Delete Agent Engine backing LRO: projects/879055303739/locations/us-central1/operations/7496070433901379584
Agent Engine deleted. Resource name: projects/879055303739/locations/us-central1/reasoningEngines/2370520681205989376
